<a href="https://colab.research.google.com/github/kridtapon/WFO-CTS-DPO-Power/blob/main/WFO_CTS_DPO_Power.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.8 MB/s eta 0:00:00


In [7]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import itertools

# Function to calculate the Chande Trend Score (CTS)
def calculate_cts(data, period=14):
    """
    Calculate the Chande Trend Score (CTS).
    """
    price_change = data['Close'].diff(periods=period)
    avg_price_change = price_change.rolling(window=period).mean()
    avg_true_range = data['High'].subtract(data['Low']).rolling(window=period).mean()
    trend_score = (avg_price_change / avg_true_range) * 100
    return trend_score

# Function to calculate the Detrended Price Oscillator (DPO)
def calculate_dpo(data, period=14):
    """
    Calculate the Detrended Price Oscillator (DPO).
    """
    sma = data['Close'].rolling(window=period).mean()
    dpo = data['Close'] - sma
    return dpo

# Walk-forward optimization with CTS and DPO
def walk_forward_optimization(df, start_year, end_year):
    results = []

    # Define dynamic ranges for parameters
    cts_period_range = range(5, 31)  # CTS period range
    dpo_period_range = range(5, 31)  # DPO period range

    for test_year in range(start_year + 4, end_year + 1):
        train_start = test_year - 4
        train_end = test_year - 1
        test_start = test_year

        train_data = df[(df.index.year >= train_start) & (df.index.year <= train_end)]
        test_data = df[df.index.year == test_year]

        best_params = None
        best_performance = -np.inf

        # Use itertools to loop through all combinations dynamically
        for params in itertools.product(cts_period_range, dpo_period_range):
            cts_period, dpo_period = params

            # Calculate indicators on the training data
            train_data['CTS'] = calculate_cts(train_data, period=cts_period)
            train_data['DPO'] = calculate_dpo(train_data, period=dpo_period)

            # Generate entry and exit signals based on CTS and DPO
            entries = (train_data['CTS'] > 30) & (train_data['DPO'] > 0)  # CTS above 30 and DPO positive
            exits = (train_data['CTS'] < -30) & (train_data['DPO'] < 0)  # CTS below -30 and DPO negative

            # Backtest on training data
            portfolio = vbt.Portfolio.from_signals(
                close=train_data['Close'],
                entries=entries,
                exits=exits,
                init_cash=100_000,
                fees=0.001
            )

            performance = portfolio.total_return()
            if performance > best_performance:
                best_performance = performance
                best_params = (cts_period, dpo_period)

        # Test with the best parameters on the test data
        test_data['CTS'] = calculate_cts(test_data, period=best_params[0])
        test_data['DPO'] = calculate_dpo(test_data, period=best_params[1])

        entries = (test_data['CTS'] > 30) & (test_data['DPO'] > 0)  # CTS above 30 and DPO positive
        exits = (test_data['CTS'] < -30) & (test_data['DPO'] < 0)  # CTS below -30 and DPO negative

        portfolio = vbt.Portfolio.from_signals(
            close=test_data['Close'],
            entries=entries,
            exits=exits,
            init_cash=100_000,
            fees=0.001
        )

        results.append({
            'Year': test_year,
            'Best_Params': best_params
        })

    return pd.DataFrame(results)

# Define the stock symbol and time period
symbol = 'KKR'
start_date = '2015-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Perform walk-forward optimization
results = walk_forward_optimization(df, 2016, 2025)

# Display results
print("\nWalk-Forward Optimization Results:")
print(results)

# Combine signals into a single portfolio
combined_entries = pd.Series(False, index=df.index)
combined_exits = pd.Series(False, index=df.index)

for _, row in results.iterrows():
    year = row['Year']
    params = row['Best_Params']
    yearly_data = df[df.index.year == year]

    # Apply CTS and DPO indicators
    yearly_data['CTS'] = calculate_cts(yearly_data, period=params[0])
    yearly_data['DPO'] = calculate_dpo(yearly_data, period=params[1])

    # Define entry/exit conditions
    entries = (yearly_data['CTS'] > 30) & (yearly_data['DPO'] > 0)
    exits = (yearly_data['CTS'] < -30) & (yearly_data['DPO'] < 0)

    combined_entries.loc[entries.index] = entries
    combined_exits.loc[exits.index] = exits

# Filter data for testing period only
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]
combined_entries = combined_entries[(combined_entries.index.year >= 2020) & (combined_entries.index.year <= 2025)]
combined_exits = combined_exits[(combined_exits.index.year >= 2020) & (combined_exits.index.year <= 2025)]

# Backtest using the combined signals
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=combined_entries,
    exits=combined_exits,
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
<ipython-input-7-1d0ca9675d77>:51: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-7-1d0ca9675d77>:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-7-1d0ca9675d77>:51: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in


Walk-Forward Optimization Results:
   Year Best_Params
0  2020    (16, 11)
1  2021     (15, 8)
2  2022     (15, 8)
3  2023      (9, 5)
4  2024      (9, 5)
5  2025      (9, 6)
Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           267735.351498
Total Return [%]                       167.735351
Benchmark Return [%]                   432.027355
Max Gross Exposure [%]                      100.0
Total Fees Paid                       5881.822676
Max Drawdown [%]                        43.057894
Max Drawdown Duration                       737.0
Total Trades                                   18
Total Closed Trades                            18
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            44.444444
Best Trade [%]          

<ipython-input-7-1d0ca9675d77>:121: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-7-1d0ca9675d77>:122: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-7-1d0ca9675d77>:121: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-7-1d0ca

In [8]:
# Buy and Hold Performance Metrics
df_holding = df['Close']
pf = vbt.Portfolio.from_holding(df_holding, init_cash=100_000)
pf.stats()

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



,Close
Start,2020-01-02 00:00:00
End,2024-12-31 00:00:00
Period,1258
Start Value,100000.0
End Value,532027.354587
Total Return [%],432.027355
Benchmark Return [%],432.027355
Max Gross Exposure [%],100.0
Total Fees Paid,0.0
Max Drawdown [%],47.915483
